In [1]:
%load_ext autoreload
%load_ext autotime
%autoreload 2

time: 689 µs (started: 2021-11-22 21:59:09 -08:00)


# Options Apps Project

## Goal
Creating a suite of equity option web apps which expands on, and was inspired by, [Harshit Tyagi’s](https://www.linkedin.com/in/tyagiharshit/) [article](https://www.kdnuggets.com/2021/09/-structured-financial-newsfeed-using-python-spacy-and-streamlit.html) on building a financial newsfeed app.

## Skills Involved
Web scraping, NLP, options price modeling, predictive modeling of volitility, and building apps with Streamlit.

## Imports

In [2]:
import sys, os, math
import requests
from bs4 import BeautifulSoup
import spacy
from spacy import displacy
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import time

#  setting path
gparent = os.path.join(os.pardir)
sys.path.append(gparent)

from src import helper_functions as h

time: 14.9 s (started: 2021-11-22 21:59:09 -08:00)


## requests: Grabbing Headlines

In [3]:
# grabbing headlines
r = requests.get("http://feeds.marketwatch.com/marketwatch/topstories/")
r

<Response [200]>

time: 226 ms (started: 2021-11-22 21:59:24 -08:00)


## bs4: Saving Headlines

In [4]:
# saving headlines to a list
soup = BeautifulSoup(r.content, features='lxml')
headlines = soup.findAll('title')
print(f'{type(headlines)}\n')
print(headlines)

<class 'bs4.element.ResultSet'>

[<title>MarketWatch.com - Top Stories</title>, <title>MarketWatch.com - Top Stories</title>, <title>The Moneyist: ‘I’d prefer the money never hits our account’: My in-laws live with my wife and I. They want to use their 401(k) to pay off our mortgage. What should we do?</title>, <title>: Americans increasingly say they don’t plan to have kids — this is the No. 1 reason why</title>, <title>: Don’t expect U.S. to follow Europe with new COVID lockdowns</title>, <title>The Wall Street Journal: Samsung set to announce $17 billion chip plant in Texas</title>, <title>Bond Report: Treasury yields see sharpest one-day rise in nearly 2 weeks as Biden picks Powell for second Fed term, named Brainard No. 2</title>, <title>The Fed: Fed’s Bostic says good arguments can be made to consider a faster taper</title>, <title>The Fed: After letting suspense build, Biden taps Powell for second term</title>, <title>: 5 things NOT to buy on Black Friday</title>, <title>: What 

### Grabbing Headline String
Grabbing headline and printing headline and isolated text.

In [5]:
# grabbing test headline
test_headline = headlines[2]
print(f'{test_headline}\n')
print(test_headline.text)

<title>The Moneyist: ‘I’d prefer the money never hits our account’: My in-laws live with my wife and I. They want to use their 401(k) to pay off our mortgage. What should we do?</title>

The Moneyist: ‘I’d prefer the money never hits our account’: My in-laws live with my wife and I. They want to use their 401(k) to pay off our mortgage. What should we do?
time: 921 µs (started: 2021-11-22 21:59:24 -08:00)


## spaCy: Tests
Testing tokenization and name extraction on a single string.

### Loading Model

In [6]:
# loading spacy model
nlp = spacy.load('en_core_web_sm')

time: 701 ms (started: 2021-11-22 21:59:24 -08:00)


### Tokenizing String
Tokenizing the test headline.

In [7]:
# checking the test case
processed_hline = nlp(test_headline.text)
print(f'{test_headline}\n')
for token in processed_hline:
  print(token)

<title>The Moneyist: ‘I’d prefer the money never hits our account’: My in-laws live with my wife and I. They want to use their 401(k) to pay off our mortgage. What should we do?</title>

The
Moneyist
:
‘
I
’d
prefer
the
money
never
hits
our
account
’
:
My
in
-
laws
live
with
my
wife
and
I.
They
want
to
use
their
401(k
)
to
pay
off
our
mortgage
.
What
should
we
do
?
time: 16 ms (started: 2021-11-22 21:59:25 -08:00)


## Saving List of Tokenized Headlines

In [8]:
processed_hlines = [nlp(headlines[i].text) for i in range(len(headlines))]
for line in processed_hlines:
    print(line)

MarketWatch.com - Top Stories
MarketWatch.com - Top Stories
The Moneyist: ‘I’d prefer the money never hits our account’: My in-laws live with my wife and I. They want to use their 401(k) to pay off our mortgage. What should we do?
: Americans increasingly say they don’t plan to have kids — this is the No. 1 reason why
: Don’t expect U.S. to follow Europe with new COVID lockdowns
The Wall Street Journal: Samsung set to announce $17 billion chip plant in Texas
Bond Report: Treasury yields see sharpest one-day rise in nearly 2 weeks as Biden picks Powell for second Fed term, named Brainard No. 2
The Fed: Fed’s Bostic says good arguments can be made to consider a faster taper
The Fed: After letting suspense build, Biden taps Powell for second term
: 5 things NOT to buy on Black Friday
: What a Fed led by Powell and Brainard means for Americans’ bank accounts
: Walt Disney World pauses sales of most annual passes — what travelers need to know
time: 85.4 ms (started: 2021-11-22 21:59:25 -08:

## Getting Org Names From Headlines
Visualizing named entities (real world objects) in the headlines and creating a set of organiziations.

In [9]:
# pulling company name tokens from headlines
companies = []
for title in processed_hlines:
    doc = nlp(title.text)
    if len(doc.ents) != 0:
        displacy.render(doc, style='ent')
    else:
        pass
    for token in doc.ents:
        if token.label_ == 'ORG':
            companies.append(token.text)
        else:
            pass
companies = set(companies)
print(companies)

{'Bostic', 'COVID', 'Walt Disney World', 'Fed', 'The Wall Street Journal', 'Treasury', 'Samsung'}
time: 114 ms (started: 2021-11-22 21:59:25 -08:00)


## Scraping S&P 500 Stock Table w/Requests & BeautifulSoup

In [10]:
# scraping S&P wikipedia page
r = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# parsing the html
soup = BeautifulSoup(r.text, 'lxml')

# extracting the table
table = soup.find('table', {'class': 'wikitable sortable'})

# printing row with first stock
print(table.findAll('tr')[1:2])

[<tr>
<td><a class="external text" href="https://www.nyse.com/quote/XNYS:MMM" rel="nofollow">MMM</a>
</td>
<td><a href="/wiki/3M" title="3M">3M</a></td>
<td><a class="external text" href="https://www.sec.gov/edgar/browse/?CIK=66740" rel="nofollow">reports</a></td>
<td>Industrials</td>
<td>Industrial Conglomerates</td>
<td><a href="/wiki/Saint_Paul,_Minnesota" title="Saint Paul, Minnesota">Saint Paul, Minnesota</a></td>
<td>1976-08-09</td>
<td>0000066740</td>
<td>1902
</td></tr>]
time: 591 ms (started: 2021-11-22 21:59:25 -08:00)


### Ticker Symbols
Grabbing the ticker symbol from the first cell of each row.

In [11]:
# making list of symbols
symbols = [row.findAll('td')[0].text for row in table.findAll('tr')[1:]]

# checking length
print(f'List Length: {len(symbols)} \n')

# checking first 5 symbols
print(f'First five symbols: {symbols[:5]}')

List Length: 505 

First five symbols: ['MMM\n', 'ABT\n', 'ABBV\n', 'ABMD\n', 'ACN\n']
time: 14.9 ms (started: 2021-11-22 21:59:26 -08:00)


The length of the list looks good, but we have extra characters we need to strip from the symbols.

In [12]:
# stripping new line character from the strings 
symbols = list(map(lambda s: s.strip(), symbols))

# checking first 5 symbols
print(symbols[:5])

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN']
time: 948 µs (started: 2021-11-22 21:59:26 -08:00)


Looks good. We can apply the same technique to the second cell of the row to scrape and save the company name, and to the fifth cell to grab the company industry. 

### Names
Grabbing the company name from the second cell of each row.

In [13]:
# making list of names
names = [row.findAll('td')[1].text for row in table.findAll('tr')[1:]]

# checking first five names
print(names[:5])

['3M', 'Abbott Laboratories', 'AbbVie', 'Abiomed', 'Accenture']
time: 14.1 ms (started: 2021-11-22 21:59:26 -08:00)


## S&P 500 Dataframe
Creating a data frame of stocks in the S&P 500 index.

In [14]:
# making a data dictionary
data = {'Company Name': names, 'Symbol': symbols}

# creating data frame from the data
stocks_df = pd.DataFrame.from_dict(data)

# checking shape and first five rows
print(stocks_df.shape)
stocks_df.head()

(505, 2)


,Company Name,Symbol
0,3M,MMM
1,Abbott Laboratories,ABT
2,AbbVie,ABBV
3,Abiomed,ABMD
4,Accenture,ACN


time: 16 ms (started: 2021-11-22 21:59:26 -08:00)


### Checking yf Stock Info Dictionary Keys
Checking the dictionary keys available for pulling stock information.

In [15]:
# instantiating a ticker object
ACN = yf.Ticker('ACN')

time: 2.96 ms (started: 2021-11-22 21:59:26 -08:00)


In [16]:
# checking keys
ACN.info.keys()

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'country', 'companyOfficers', 'website', 'maxAge', 'address1', 'fax', 'industry', 'address2', 'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 'operatingMargins', 'ebitda', 'targetLowPrice', 'recommendationKey', 'grossProfits', 'freeCashflow', 'targetMedianPrice', 'currentPrice', 'earningsGrowth', 'currentRatio', 'returnOnAssets', 'numberOfAnalystOpinions', 'targetMeanPrice', 'debtToEquity', 'returnOnEquity', 'targetHighPrice', 'totalCash', 'totalDebt', 'totalRevenue', 'totalCashPerShare', 'financialCurrency', 'revenuePerShare', 'quickRatio', 'recommendationMean', 'exchange', 'shortName', 'longName', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'isEsgPopulated', 'gmtOffSetMilliseconds', 'quoteType', 'symbol', 'messageBoardId', 'market', 'annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskR

time: 2.83 s (started: 2021-11-22 21:59:26 -08:00)


### Data for Stocks in the News
Creating a data frame of price and dividend information for S&P 500 stocks in the news in the following steps:

- Create a data dictionary from the list of companies in the headlines.

- Create a data frame from data dictionary.

In [17]:
# creating empty stock info dictionary
stock_data = {
    'Company': [],
    'Symbol': [],
    'currentPrice': [],
    'dayHigh': [],
    'dayLow': [],
    '52wkHigh': [],
    '52wkLow': [],
    'dividendRate': []
    
}

# loading stocks from s&p dataframe and appending data from yf
for company in companies:

    try:
        if stocks_df['Company Name'].str.contains(company).sum():
            symbol = stocks_df[stocks_df['Company Name'].\
                                str.contains(company)]['Symbol'].values[0]
            org_name = stocks_df[stocks_df['Company Name'].\
                                str.contains(company)]['Company Name'].values[0]
            stock_data['Company'].append(org_name)
            stock_data['Symbol'].append(symbol)
            stock_info = yf.Ticker(symbol).info
            stock_data['currentPrice'].append(stock_info['currentPrice'])
            stock_data['dayHigh'].append(stock_info['dayHigh'])
            stock_data['dayLow'].append(stock_info['dayLow'])
            stock_data['52wkHigh'].append(stock_info['fiftyTwoWeekHigh'])
            stock_data['52wkLow'].append(stock_info['fiftyTwoWeekLow'])            
            
            # converting dividend None types to floats
            dividend = stock_info['dividendRate']
            if dividend != None:
                dividend = dividend
            else:
                dividend = 0
            stock_data['dividendRate'].append(dividend)
        else:
            pass
    except:
        pass

time: 3.54 s (started: 2021-11-22 21:59:29 -08:00)


In [18]:
# checking dict
stock_data

{'Company': ['Federal Realty Investment Trust'],
 'Symbol': ['FRT'],
 'currentPrice': [129.85],
 'dayHigh': [130.765],
 'dayLow': [128.29],
 '52wkHigh': [135.56],
 '52wkLow': [81.85],
 'dividendRate': [4.28]}

time: 2.03 ms (started: 2021-11-22 21:59:32 -08:00)


### Data Frame of S&P 500 Stocks in the News

In [19]:
in_the_news = pd.DataFrame(stock_data)
in_the_news.head()

,Company,Symbol,currentPrice,dayHigh,dayLow,52wkHigh,52wkLow,dividendRate
0,Federal Realty Investment Trust,FRT,129.85,130.765,128.29,135.56,81.85,4.28


time: 6.75 ms (started: 2021-11-22 21:59:33 -08:00)


In [20]:
# checking Dtypes
in_the_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Company       1 non-null      object 
 1   Symbol        1 non-null      object 
 2   currentPrice  1 non-null      float64
 3   dayHigh       1 non-null      float64
 4   dayLow        1 non-null      float64
 5   52wkHigh      1 non-null      float64
 6   52wkLow       1 non-null      float64
 7   dividendRate  1 non-null      float64
dtypes: float64(6), object(2)
memory usage: 192.0+ bytes
time: 5.14 ms (started: 2021-11-22 21:59:33 -08:00)


## Individual Stock Price Helper Function
Checking the helper function.

In [21]:
h.get_prices('MMM')

,Company,Symbol,Current Price,Intraday High,Intraday Low,52wkHigh,52wkLow,Dividend
0,3M Company,MMM,179.66,182.28,178.78,208.95,163.38,5.92


time: 2.44 s (started: 2021-11-22 21:59:33 -08:00)


In [22]:
h.get_prices('AAPL')

,Company,Symbol,Current Price,Intraday High,Intraday Low,52wkHigh,52wkLow,Dividend
0,Apple Inc.,AAPL,161.02,165.7,161,165.7,112.59,0.88


time: 4.8 s (started: 2021-11-22 21:59:35 -08:00)


## Options Markets Tests

In [23]:
# creating a fuction to display available expirations
def get_expirations(symbol : str):
    stock = yf.Ticker(symbol)
    return stock.options

time: 838 µs (started: 2021-11-22 21:59:40 -08:00)


In [24]:
get_expirations('AAPL')

('2021-11-26',
 '2021-12-03',
 '2021-12-10',
 '2021-12-17',
 '2021-12-23',
 '2021-12-31',
 '2022-01-21',
 '2022-02-18',
 '2022-03-18',
 '2022-04-14',
 '2022-05-20',
 '2022-06-17',
 '2022-07-15',
 '2022-09-16',
 '2023-01-20',
 '2023-03-17',
 '2023-06-16',
 '2023-09-15',
 '2024-01-19')

time: 169 ms (started: 2021-11-22 21:59:40 -08:00)


In [25]:
get_expirations('MMM')

('2021-11-26',
 '2021-12-03',
 '2021-12-10',
 '2021-12-17',
 '2021-12-23',
 '2021-12-31',
 '2022-01-21',
 '2022-04-14',
 '2022-06-17',
 '2022-07-15',
 '2023-01-20',
 '2024-01-19')

time: 130 ms (started: 2021-11-22 21:59:40 -08:00)


In [26]:
def options_mrkt_test(symbol : str, expiration : str, option : str):
    stock = yf.Ticker(symbol)
    current_price = stock.info['currentPrice']
    dividend = stock.info['dividendRate']
    url = ('https://www.treasury.gov/resource-center/'
    'data-chart-center/interest-rates/Pages/TextView.aspx?data=yield')
    rates = pd.read_html(url)
    risk_free_rate = rates[1]['3 mo']
    print(f"Current Price: ${current_price}")
    print(f"Dividend: ${dividend}")
    print(f"3 Month TBill Rate: {risk_free_rate.to_string().split()[1]}%")
    opt = stock.option_chain(expiration)
    if option == 'calls':
        return opt.calls.sort_values(by='strike')
    elif option == 'puts':
        return opt.puts.sort_values(by='strike')

time: 1.17 ms (started: 2021-11-22 21:59:40 -08:00)


In [28]:
options_mrkt_test('MMM', '2021-11-26', 'calls' )

Current Price: $179.66
Dividend: $5.92
3 Month TBill Rate: 0.05%


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,MMM211126C00170000,2021-11-22 19:11:08,170.0,11.96,0.0,0.0,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
1,MMM211126C00175000,2021-11-22 17:54:25,175.0,6.25,0.0,0.0,0.0,0.0,1.0,0,0.000010,True,REGULAR,USD
2,MMM211126C00177500,2021-11-22 16:31:35,177.5,4.60,0.0,0.0,0.0,0.0,31.0,0,0.000010,True,REGULAR,USD
3,MMM211126C00180000,2021-11-22 20:57:47,180.0,1.26,0.0,0.0,0.0,0.0,140.0,0,0.007822,False,REGULAR,USD
4,MMM211126C00182500,2021-11-22 20:59:16,182.5,0.40,0.0,0.0,0.0,0.0,332.0,0,0.031260,False,REGULAR,USD
5,MMM211126C00185000,2021-11-22 20:50:30,185.0,0.17,0.0,0.0,0.0,0.0,615.0,0,0.062509,False,REGULAR,USD
6,MMM211126C00187500,2021-11-22 20:47:55,187.5,0.06,0.0,0.0,0.0,0.0,219.0,0,0.125009,False,REGULAR,USD
7,MMM211126C00190000,2021-11-22 20:53:21,190.0,0.04,0.0,0.0,0.0,0.0,55.0,0,0.125009,False,REGULAR,USD
8,MMM211126C00192500,2021-11-22 15:32:59,192.5,0.03,0.0,0.0,0.0,0.0,3.0,0,0.125009,False,REGULAR,USD
9,MMM211126C00195000,2021-11-22 17:13:10,195.0,0.04,0.0,0.0,0.0,0.0,13.0,0,0.125009,False,REGULAR,USD


time: 4.12 s (started: 2021-11-22 22:00:15 -08:00)
